In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sqlite3
from datetime import datetime 
import random
import json

## Models

In [ ]:
intents_entities = {
    "change_first_name": ["matriculation_number", "first_name"],
    "change_last_name": ["matriculation_number", "last_name"],
    "change_address": ["matriculation_number", "address", "city", "post_code"],
    "register_exam": ["matriculation_number", "course_name"],
    "deregister_exam": ["matriculation_number", "course_name"],
    "query_exam_status": ["matriculation_number", "course_name"],
    "query_exam_grade": ["matriculation_number", "course_name"],
    "receive_exam_schedule": ["matriculation_number"]
}

### LSTM

In [ ]:
model = load_model('../models/intent_lstm_model.keras')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

with open('../models/tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)
with open('../models/label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

def lstm_classify_intent(text):
    text = text.lower()
    
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=model.input_shape[1], padding='post')
    
    prediction = model.predict(padded_sequence)
    
    predicted_index = np.argmax(prediction, axis=1)[0]
    classified_intent = label_encoder.inverse_transform([predicted_index])[0]
    confidence = np.max(prediction)
    
    return classified_intent, confidence

In [ ]:
test_text = "Good afternoon! My name is Emma, and my ID is 1234567. The exam session starts next week. What is my registration status for Computer Networks exam?"
classified_intent, confidence = lstm_classify_intent(test_text)
if (confidence >= 0.9):
    print(f"Classified Intent: {classified_intent}, Confidence: {confidence:.3f}")
else:
    print(f"Unknown Intent, Confidence: {confidence:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Classified Intent: query_exam_status, Confidence: 0.996


### NER Model

In [ ]:
import string
nlp = spacy.load("../models/custom_ner_model")

test_text = "Good afternoon! My name is Alice, and my ID is 1234567. The exam session starts next week. What is my registration status for Computer Networks exam?"
doc = nlp(test_text)

for ent in doc.ents:
    clean_text = ent.text.strip(string.punctuation)
    print(f"{ent.text}: {ent.label_}")

Alice: last_name
1234567.: matriculation_number
Computer Networks: course_name


### Classification

In [ ]:
def find_missed_entities(classified_intent, detected_entities):
    print(f'LOG find_missed_entities: classified_intent {classified_intent}, detected_entities {detected_entities}')
    required_entities = intents_entities.get(classified_intent, [])
    missed_entities = [entity for entity in required_entities if entity not in detected_entities]
    return missed_entities

def chatbot_classification_with_entity_validation(user_input):
    print(f'LOG chatbot_classification_with_entity_validation: user_input {user_input}')
    classified_intent, confidence = lstm_classify_intent(user_input)

    nlp = spacy.load("../models/custom_ner_model")

    doc = nlp(user_input)
    detected_entities = {}
    for ent in doc.ents:
        detected_entities[ent.label_] = ent.text

    missed_entities = find_missed_entities(classified_intent, detected_entities)

    return classified_intent, confidence, detected_entities, missed_entities

In [ ]:
user_input = "Good morning, my name is Meier (123456) and I've moved. I now live in 44143 Dortmund at Hauptstrasse 12. Please change my address."
classified_intent, confidence, detected_entities, missed_entities = chatbot_classification_with_entity_validation(user_input)


if (confidence >= 0.985):
    print(f"Classified Intent: {classified_intent}, Confidence: {confidence:.3f}")
else:
    print(f"Unknown Intent, Confidence: {confidence:.3f}")
print(f"Detected Entities: {detected_entities}")
print(f"Missed Entities: {missed_entities}")

LOG chatbot_classification_with_entity_validation: user_input Good morning, my name is Meier (123456) and I've moved. I now live in 44143 Dortmund at Hauptstrasse 12. Please change my address.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
LOG find_missed_entities: classified_intent change_address, detected_entities {'post_code': '44143', 'city': 'Dortmund', 'address': 'Hauptstrasse 12.'}
Classified Intent: change_address, Confidence: 0.999
Detected Entities: {'post_code': '44143', 'city': 'Dortmund', 'address': 'Hauptstrasse 12.'}
Missed Entities: ['matriculation_number']


## Database Manipulation

In [ ]:
connection = sqlite3.connect('../database/shared_database.db')
cursor = connection.cursor()
cursor

In [ ]:
matriculation_number = '1234567'
cursor.execute('SELECT * FROM students WHERE matriculation_number = ?', (matriculation_number,))
print(cursor.fetchall())

[(1, 'Jurgen', 'Muller', '1234567')]


In [ ]:
cursor.execute('SELECT * FROM student_address WHERE id = 1')
print(cursor.fetchall())

[(1, 'Am Gardenkamp 51', 'Dortmund', '44227', 1)]


In [ ]:
def change_first_name(first_name, matriculation_number):
    print(f'LOG change_first_name: Change the first name to {first_name} for {matriculation_number}')
    try:
        cursor.execute('SELECT id FROM students WHERE matriculation_number = ?', (matriculation_number,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return "Student not found"
        cursor.execute('UPDATE students SET first_name = ? WHERE matriculation_number = ?', (first_name, matriculation_number))
        connection.commit()
        
        return f"Your first name changed to {first_name} successfully"
    except Exception as e:
        return f"An error occurred while changing the first name: {str(e)}"

def change_last_name(last_name, matriculation_number):
    print(f'LOG change_last_name: Change the last name to {last_name} for {matriculation_number}')
    try:
        cursor.execute('SELECT id FROM students WHERE matriculation_number = ?', (matriculation_number,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return "Student not found"
        cursor.execute('UPDATE students SET last_name = ? WHERE matriculation_number = ?', (last_name, matriculation_number))
        connection.commit()

        return f"Your last name changed to {last_name} successfully"
    except Exception as e:
        return f"An error occurred while changing the last name: {str(e)}"

def change_address(address, city, post_code, matriculation_number):
    print(f'LOG change_address: Change address {address}, city {city}, post_code {post_code}, for {matriculation_number}')
    try:
        cursor.execute('SELECT id FROM students WHERE matriculation_number = ?', (matriculation_number,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return "Student not found"
        student_id = student_id_result[0]
        cursor.execute('UPDATE student_address SET address = ?, city = ?, post_code = ? WHERE student_id = ?', 
                    (address, city, post_code, student_id))
        connection.commit()

        return f"Your address changed to {address}, {city} {post_code} successfully"
    except Exception as e:
        return f"An error occurred while changing address: {str(e)}"
    
def register_exam(course_name, matriculation_number):
    print(f'LOG register_exam: Register {matriculation_number} for exam {course_name}')
    try:
        cursor.execute('SELECT id FROM students WHERE matriculation_number = ?', (matriculation_number,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return f"Student with matriculation number {matriculation_number} is not found"
        student_id = student_id_result[0]

        cursor.execute('SELECT id FROM courses WHERE name = ?', (course_name,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return f"Course with name {course_name} is not found"
        course_id = student_id_result[0]

        cursor.execute('SELECT id, time FROM exams WHERE course_id = ?', (course_id,))
        exam_result = cursor.fetchone()
        if not exam_result:
            return f"{course_name} exam is not found"
        exam_id, exam_time = exam_result

        exam_datetime = datetime.strptime(exam_time, '%Y-%m-%d %H:%M:%S')
        current_time = datetime.now()

        if exam_datetime < current_time:
            return f"{course_name} exam already completed on {exam_time}"

        cursor.execute('INSERT INTO student_exams (student_id, exam_id) VALUES (?, ?)', (student_id, exam_id))
        connection.commit()

        return f"You registered for {course_name} exam successfully"
    except Exception as e:
        return f"An error occurred while registering for the exam: {str(e)}"

def deregister_exam(course_name, matriculation_number):
    print(f'LOG deregister_exam: Deregister {matriculation_number} from exam {course_name}')
    try:
        cursor.execute('SELECT id FROM students WHERE matriculation_number = ?', (matriculation_number,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return f"Student with matriculation number {matriculation_number} is not found"
        student_id = student_id_result[0]

        cursor.execute('SELECT id FROM courses WHERE name = ?', (course_name,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return f"Course with name {course_name} is not found"
        course_id = student_id_result[0]

        cursor.execute('SELECT id, time FROM exams WHERE course_id = ?', (course_id,))
        exam_id_result = cursor.fetchone()
        if not exam_id_result:
            return f"{course_name} exam is not found"
        exam_id, exam_time = exam_id_result

        exam_datetime = datetime.strptime(exam_time, '%Y-%m-%d %H:%M:%S')
        current_time = datetime.now()

        if exam_datetime < current_time:
            return f"{course_name} exam already completed on {exam_time}"

        cursor.execute('DELETE student_exams where student_id = ? and exam_id = ?', (student_id, exam_id))
        connection.commit()

        return f"You deregistered from {course_name} exam successfully"
    except Exception as e:
        return f"An error occurred while dereginstering from the exam: {str(e)}"

def query_exam_status(course_name, matriculation_number):
    print(f'LOG query_exam_status: Query exam status {matriculation_number} for exam {course_name}')
    try:
        cursor.execute('SELECT id FROM students WHERE matriculation_number = ?', (matriculation_number,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return f"Student with matriculation number {matriculation_number} is not found"
        student_id = student_id_result[0]

        cursor.execute('SELECT id FROM courses WHERE name = ?', (course_name,))
        course_id_result = cursor.fetchone()
        if not course_id_result:
            return f"Course with name {course_name} is not found"
        course_id = course_id_result[0]

        cursor.execute('SELECT id, time FROM exams WHERE course_id = ?', (course_id,))
        exam_result = cursor.fetchone()
        if not exam_result:
            return f"{course_name} exam is not found"
        exam_id, exam_time = exam_result

        cursor.execute('SELECT * FROM student_exams WHERE exam_id = ? AND student_id = ?', (exam_id, student_id))
        student_exam_result = cursor.fetchone()
        if not student_exam_result:
            return f"Your status for {course_name} is Not Registered"
        
        exam_datetime = datetime.strptime(exam_time, '%Y-%m-%d %H:%M:%S')
        current_time = datetime.now()

        if exam_datetime > current_time:
            return f"Your status for {course_name} is Registered"
        else:
            return f"Examination for {course_name} already completed"
    except Exception as e:
        return f"An error occurred while quering the exam status: {str(e)}"


def query_exam_grade(course_name, matriculation_number):
    print(f'LOG query_exam_grade: Query grade {matriculation_number} for exam {course_name}')
    
    try:
        cursor.execute('SELECT id FROM students WHERE matriculation_number = ?', (matriculation_number,))
        student_id_result = cursor.fetchone()
        if not student_id_result:
            return f"Student with matriculation number {matriculation_number} is not found"
        student_id = student_id_result[0]

        cursor.execute('SELECT id FROM courses WHERE name = ?', (course_name,))
        course_id_result = cursor.fetchone()
        if not course_id_result:
            return f"Course with name {course_name} is not found"
        course_id = course_id_result[0]

        cursor.execute('SELECT id FROM exams WHERE course_id = ?', (course_id,))
        exam_id_result = cursor.fetchone()
        if not exam_id_result:
            return f"{course_name} exam is not found"
        exam_id = exam_id_result[0]

        cursor.execute('SELECT grade FROM student_exam_grades WHERE exam_id = ? AND student_id = ?', (exam_id, student_id))
        student_exam_grade_result = cursor.fetchone()
        if not student_exam_grade_result:
            return "Exam is not passed"
        student_exam_grade = student_exam_grade_result[0]
        return f"Your grade for {course_name} exam is {student_exam_grade}"
    except Exception as e:
        return f"An error occurred while quering the exam grade: {str(e)}"

In [ ]:
query_exam_grade('Artificial Intelligence', '1234567')

LOG query_exam_grade: Query grade 1234567 for exam Artificial Intelligence


'Your grade for Artificial Intelligence exam is A'

In [ ]:
actions = {
    "change_first_name": change_first_name,
    "change_last_name": change_last_name,
    "change_address": change_address,
    "register_exam": register_exam,
    "deregister_exam": deregister_exam,
    "query_exam_status": query_exam_status,
    "query_exam_grade": query_exam_grade
}

In [ ]:
reading_database = ["query_exam_status", "query_exam_grade"]
writing_database = ["change_first_name", "change_last_name", "change_address", "register_exam", "deregister_exam", "change_last_name"]

In [ ]:
human_readable_entity = {
    'matriculation_number': "matriculation number",
    'course_name': 'course name',
    'address': 'address',
    'city': 'city',
    'post_code': 'post code',
    'first_name': 'first name',
    'last_name': 'last name'
}

In [ ]:
def check_missing_entities(missed_entities):
    print(f'LOG: check_missing_entities: {missed_entities}')
    if missed_entities:
        missing_info = ', '.join([human_readable_entity[entity] for entity in missed_entities])
        return f"Missing information: {missing_info}. Please provide the required details."
    
def get_action_by_intent(classified_intent):
    print(f'LOG: get_action_by_intent: {classified_intent}')
    action = actions.get(classified_intent)
    if not action:
        return "Sorry, I couldn't identify the correct action to take.", False
    return action, True

def process_user_request(action, detected_entities):
    print(f'LOG: process_user_request: action {action}, detected_entities {detected_entities}')
    try:
        required_entities = intents_entities.get(action.__name__, [])
        
        action_arguments = {
            key: value for key, value in detected_entities.items() if key in required_entities
        }

        print(f'LOG: process_user_request: action_arguments {action_arguments}')
        return(action(**action_arguments))
    except Exception as e:
        return f"An error occurred while performing the operation: {str(e)}"

In [ ]:
user_input = "What is my grade for Artificial Intelligence exam?"
classified_intent, confidence, detected_entities, missed_entities = chatbot_classification_with_entity_validation(user_input)

if (confidence >= 0.7):
    print(f"Classified Intent: {classified_intent}, Confidence: {confidence:.3f}")
    print(f"Detected Entities: {detected_entities}")
    print(f"Missed Entities: {missed_entities}")
else:
    classified_intent = "unknown_intent"
    print(f"Unknown Intent, Confidence: {confidence:.3f}")

check = check_missing_entities(missed_entities)

if not check:
    action, check = get_action_by_intent(classified_intent)
    print(action)
    if (check):
        response = process_user_request(action, detected_entities)
        print(response)

LOG chatbot_classification_with_entity_validation: user_input What is my grade for Artificial Intelligence exam?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
LOG find_missed_entities: classified_intent query_exam_grade, detected_entities {'course_name': 'Artificial Intelligence'}
Classified Intent: query_exam_grade, Confidence: 0.996
Detected Entities: {'course_name': 'Artificial Intelligence'}
Missed Entities: ['matriculation_number']
LOG: check_missing_entities: ['matriculation_number']


## Chat

In [ ]:
with open("../data/raw/rule_based_intents.json", "r") as file:
    rule_based_data = json.load(file)

In [ ]:
user_context = {
    "classified_intent": None,
    "detected_entities": {},
    "missed_entities": [],
    "matriculation_number": None
}

confirmation_context = {
    "summary": None,
    "classified_intent": None,
    "detected_entities": {}
}

def match_rule_based_intent(user_input, rule_based_data):
    print(f'LOG match_rule_based_intent: user_input: {user_input}, rule_based_data: {rule_based_data}')
    for intent, data in rule_based_data.items():
        for pattern in data["patterns"]:
            if pattern in user_input.lower():
                return intent
    return None

def generate_rule_based_response(intent, rule_based_data):
    print(f'LOG generate_rule_based_response: intent {intent}, rule_based_data {rule_based_data}')
    responses = rule_based_data[intent]["responses"]
    return random.choice(responses)

def chatbot_response(user_input):
    global user_context, confirmation_context

    if confirmation_context["summary"]:
        if user_input.lower() in ["yes", "y", "correct", "+", "good"]:
            action, check = get_action_by_intent(confirmation_context["predicted_intent"])
            response = process_user_request(action, confirmation_context["detected_entities"])
            confirmation_context["summary"] = None  
            return response
        elif user_input.lower() in ["no", "n", "wrong", "incorrect", "-"]:
            confirmation_context["summary"] = None  
            return "Request canceled. How else can I assist you?"
        else:
            return f"Please confirm: {confirmation_context['summary']}"

    rule_based_intent = match_rule_based_intent(user_input, rule_based_data)
    if rule_based_intent:
        return generate_rule_based_response(rule_based_intent, rule_based_data)
    
    if user_context["missed_entities"]:
        missing_entity = user_context["missed_entities"].pop(0)

        if missing_entity == "matriculation_number":
            user_context["matriculation_number"] = user_input
        user_context["detected_entities"][missing_entity] = user_input

        if not user_context["missed_entities"]:
            return process_request_with_context()
        else:
            next_missing_entity = user_context["missed_entities"][0]
            return f"Please provide the {human_readable_entity[next_missing_entity]}."

    classified_intent, confidence, detected_entities, missed_entities = chatbot_classification_with_entity_validation(user_input)

    print(f"LOG: Classified Intent: {classified_intent}, Confidence: {confidence:.3f}")
    print(f"LOG: Detected Entities: {detected_entities}")
    print(f"LOG: Missed Entities: {missed_entities}")

    if confidence >= 0.9:
        user_context["classified_intent"] = classified_intent
        user_context["detected_entities"].update(detected_entities)
        user_context["missed_entities"] = missed_entities

        if "matriculation_number" in user_context["detected_entities"]:
            user_context["matriculation_number"] = user_context["detected_entities"]["matriculation_number"]
    else:
        return "Sorry, I couldn't identify the correct action to take."

    if missed_entities:
        missing_entity = missed_entities[0]
        if missing_entity == "matriculation_number" and user_context.get("matriculation_number"):
            detected_entities["matriculation_number"] = user_context["matriculation_number"]
        else:
            return f"Missing information: {human_readable_entity[missing_entity]}. Please provide the required details."

    return summarize_and_confirm_request(classified_intent, detected_entities)

def summarize_and_confirm_request(intent, detected_entities):
    global confirmation_context

    entity_summary = ", ".join(f"{human_readable_entity[key]}: {value}" for key, value in detected_entities.items())

    if intent in reading_database:
        action, check = get_action_by_intent(intent)
        return process_user_request(action, detected_entities)
    elif intent in writing_database:
        summary = f"You want to {intent.replace('_', ' ')} with the following details: {entity_summary}. Is this correct?"
        confirmation_context.update({
            "summary": summary,
            "classified_intent": intent,
            "detected_entities": detected_entities
        })
        print('I m here')
        return summary
    else:
        return "Sorry, I couldn't identify the correct operation type."

def process_request_with_context():
    global user_context
    classified_intent = user_context["classified_intent"]
    print(f"LOG: process_request_with_context: classified_intent {classified_intent}")
    detected_entities = user_context["detected_entities"]

    print(f"LOG: process_request_with_context: action {classified_intent}, detected_entities {detected_entities}")

    response = summarize_and_confirm_request(classified_intent, detected_entities)
    
    user_context = {
        "classified_intent": None,
        "detected_entities": {},
        "missed_entities": [],
        "matriculation_number": user_context["matriculation_number"]
    }
    return response

end = True
print(">>>>>>>>>>>>>>>>>>>Chatbot: Hello! I am your university chatbot. How can I help you?")
while end:
    user_input = input("> ") 
    print(f'>>>>>>>>>>>>>>>>>>>You: {user_input}')
    if user_input.lower() in ["exit", "quit", "bye"]:
        print(">>>>>>>>>>>>>>>>>>>Chatbot: Goodbye! Have a great day!")
        end = False
    else:
        response = chatbot_response(user_input)
        print(f'>>>>>>>>>>>>>>>>>>>Chatbot: {response}')

>>>>>>>>>>>>>>>>>>>Chatbot: Hello! I am your university chatbot. How can I help you?
>>>>>>>>>>>>>>>>>>>You: What is my grade for Artificial Intelligence exam?
LOG match_rule_based_intent: user_input: What is my grade for Artificial Intelligence exam?, rule_based_data: {'greeting': {'patterns': ['hello', 'hi', 'good morning', 'good afternoon', 'hey', 'greetings'], 'responses': ['Hello! How can I assist you today?', 'Hi there! What can I do for you?']}, 'farewell': {'patterns': ['bye', 'goodbye', 'see you', 'take care', 'farewell'], 'responses': ['Goodbye! Have a great day!', 'See you later! Take care!']}, 'thanksgiving': {'patterns': ['thank you', 'thanks', 'appreciate it', 'grateful'], 'responses': ["You're welcome!", 'Glad I could help!']}, 'chatbot_identity': {'patterns': ['who are you', 'what are you', 'tell me about yourself', 'what is this', 'who am I talking to', "what's your name"], 'responses': ["I am StudentDesk, your university chatbot. I'm here to assist you with any querie